In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import time
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch

import pickle

In [2]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample

c:\Users\Revalda Putawara\.conda\envs\bnntest\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pyro.infer.autoguide import AutoDiagonalNormal

In [4]:
from tqdm import tqdm

In [5]:
import numpy as np
from sklearn.metrics import confusion_matrix

In [6]:
import copy

## Defining Model and Loading Model Training Result

In [7]:
device = torch.device("cuda")

In [8]:
class BayesianCNNSingleFC(PyroModule):
    def __init__(self, num_classes):
        super().__init__()

        prior_mu = 0.
        #prior_sigma = 0.1 #accuracy 13.203704% 2 epochs
        #prior_sigma = 1. #accuracy 31% 2 epochs
        prior_sigma = torch.tensor(10., device=device) #accuracy 45% 10 epochs
        #prior_sigma = 100 #accuracy 21% 10 epochs

        self.conv1 = PyroModule[nn.Conv2d](3, 32, kernel_size=5, stride=1, padding=2)
        self.conv1.weight = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([32, 3, 5, 5]).to_event(4))
        self.conv1.bias = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([32]).to_event(1))

        self.conv2 = PyroModule[nn.Conv2d](32, 64, kernel_size=5, stride=1, padding=2) #initially padding=1 kernel_size=3, without stride
        self.conv2.weight = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([64, 32, 5, 5]).to_event(4))
        self.conv2.bias = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([64]).to_event(1))

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = PyroModule[nn.Linear](64 * 16 * 16, num_classes)
        self.fc1.weight = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([num_classes, 64 * 16 * 16]).to_event(2))
        self.fc1.bias = PyroSample(dist.Normal(prior_mu, prior_sigma).expand([num_classes]).to_event(1))

    def forward(self, x, y=None):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        logits = self.fc1(x)
        
        # THIS IS THE MISSING PIECE: Define the likelihood
        if y is not None:
            with pyro.plate("data", x.shape[0]):
                pyro.sample("obs", dist.Categorical(logits=logits), obs=y)
        
        return logits

In [9]:
def load_data(batch_size=54):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.3444, 0.3809, 0.4082], std=[0.1809, 0.1331, 0.1137])
    ])

    dataset = datasets.EuroSAT(root='./data', transform=transform, download=False)

    torch.manual_seed(42)
    
    with open('datasplit/split_indices.pkl', 'rb') as f:
        split = pickle.load(f)
        train_dataset = Subset(dataset, split['train'])
        test_dataset = Subset(dataset, split['test'])

    # Add num_workers and pin_memory for faster data loading
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                             num_workers=4, pin_memory=True, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                            num_workers=4, pin_memory=True, persistent_workers=True)
    return train_loader, test_loader

In [10]:
num_classes = 10
bayesian_model = BayesianCNNSingleFC(num_classes=num_classes).to(device)

In [11]:
def predict_data_probs(model, test_loader, num_samples=10):
    model.eval()

    all_labels = []
    all_predictions = []
    all_logits = []
    all_probs = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)

            logits_mc = torch.zeros(num_samples, images.size(0), model.fc1.out_features).to(device)

            for i in range(num_samples):
                guide_trace = pyro.poutine.trace(guide).get_trace(images)
                replayed_model = pyro.poutine.replay(model, trace=guide_trace)
                logits = replayed_model(images)
                logits_mc[i] = logits

            avg_logits = logits_mc.mean(dim=0)
            predictions = torch.argmax(avg_logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            all_logits.extend(avg_logits.cpu().numpy())
            all_probs.extend(F.softmax(avg_logits, dim=1).cpu().numpy())

    return all_labels, all_predictions, all_logits, all_probs

## Before Bitflip

In [114]:
model_path = 'results_eurosat/bayesian_cnn_model_std10_100_epoch.pth'
guide_path = 'results_eurosat/bayesian_cnn_guide_std10_100_epoch_guide.pth'
pyro_param_store_path = 'results_eurosat/pyro_param_store_std10_100_epoch.pkl'

guide = AutoDiagonalNormal(bayesian_model).to(device)
#guide.load_state_dict(torch.load(guide_path))

pyro.get_param_store().set_state(torch.load(pyro_param_store_path,weights_only=False))

original_param_store = {}

for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")
    original_param_store[name] = torch.tensor(value.data, requires_grad=value.requires_grad)

AutoDiagonalNormal.loc: torch.Size([217546])
AutoDiagonalNormal.scale: torch.Size([217546])


C:\Users\Revalda Putawara\AppData\Local\Temp\ipykernel_14524\467444935.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  original_param_store[name] = torch.tensor(value.data, requires_grad=value.requires_grad)


In [13]:
train_loader, test_loader = load_data(batch_size=54)

In [14]:
for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")
    print(value)

AutoDiagonalNormal.loc: torch.Size([217546])
Parameter containing:
tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', requires_grad=True)
AutoDiagonalNormal.scale: torch.Size([217546])
tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<SoftplusBackward0>)


In [15]:
all_labels, all_predictions, all_logits, all_probs = predict_data_probs(bayesian_model, test_loader, num_samples=10)

Evaluating: 100%|██████████| 100/100 [00:47<00:00,  2.12it/s]


In [16]:
cm = confusion_matrix(all_labels, all_predictions)

In [17]:
#print accuracy from confusion matrix
accuracy = np.trace(cm) / np.sum(cm)
print(f"Accuracy from confusion matrix: {accuracy * 100:.6f}%")

Accuracy from confusion matrix: 74.907407%


## Check parameters

In [ ]:
original_param_store["AutoDiagonalNormal.loc"]

In [ ]:
original_param_store["AutoDiagonalNormal.scale"]

In [ ]:
for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")
    print(value)

In [ ]:
guide.loc

In [ ]:
guide.scale

In [ ]:
posterior = guide.get_posterior()
print("Before:", posterior.stddev[0].item())

In [ ]:
posterior.mean

In [ ]:
posterior.stddev

In [ ]:
import torch

def inv_softplus(x):
    return torch.log(torch.exp(x) - 1.0)

with torch.no_grad():
    scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")
    scale_param.data[-1] = inv_softplus(torch.tensor(88.7225, device=scale_param.device))

In [ ]:
scale_param

In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

In [ ]:
guide.scale

In [ ]:
# This is the approach that works (from your earlier code)
#with torch.no_grad():
#    scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")
#    scale_param.data[0] = inv_softplus(torch.tensor(100.0, device=scale_param.device))

In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

## Bitflip

In [62]:
from bitflip import bitflip_float32

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.loc"]

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.scale"]

In [18]:
param_store = pyro.get_param_store()

In [25]:
help(param_store["AutoDiagonalNormal.loc"])

Help on Parameter in module torch.nn.parameter object:

class Parameter(torch.Tensor)
 |  Parameter(data=None, requires_grad=True)
 |
 |  A kind of Tensor that is to be considered a module parameter.
 |
 |  Parameters are :class:`~torch.Tensor` subclasses, that have a
 |  very special property when used with :class:`Module` s - when they're
 |  assigned as Module attributes they are automatically added to the list of
 |  its parameters, and will appear e.g. in :meth:`~Module.parameters` iterator.
 |  Assigning a Tensor doesn't have such effect. This is because one might
 |  want to cache some temporary state, like last hidden state of the RNN, in
 |  the model. If there was no such class as :class:`Parameter`, these
 |  temporaries would get registered too.
 |
 |  Args:
 |      data (Tensor): parameter tensor.
 |      requires_grad (bool, optional): if the parameter requires gradient. Note that
 |          the torch.no_grad() context does NOT affect the default behavior of
 |          

In [24]:
help(param_store["AutoDiagonalNormal.scale"])

Help on Tensor in module torch object:

class Tensor(torch._C.TensorBase)
 |  Method resolution order:
 |      Tensor
 |      torch._C.TensorBase
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __abs__ = abs(...)
 |
 |  __array__(self, dtype=None) from torch._tensor.Tensor
 |
 |  __array_wrap__(self, array) from torch._tensor.Tensor
 |      # Wrap Numpy array again in a suitable tensor when done, to support e.g.
 |      # `numpy.sin(tensor) -> tensor` or `numpy.greater(tensor, 0) -> ByteTensor`
 |
 |  __contains__(self, element: Any, /) -> bool from torch._tensor.Tensor
 |      Check if `element` is present in tensor
 |
 |      Args:
 |          element (Tensor or scalar): element to be checked
 |              for presence in current tensor"
 |
 |  __deepcopy__(self, memo) from torch._tensor.Tensor
 |
 |  __dir__(self) from torch._tensor.Tensor
 |      Default dir() implementation.
 |
 |  __dlpack__(self, stream=None) from torch._tensor.Tensor
 |      Creates a DLpack `caps

In [19]:
param_store

In [26]:
# For loc (works directly)
#with torch.no_grad():
#    param_store["AutoDiagonalNormal.loc"].data.copy_(modified_loc_tensor)

# For scale (need to work with unconstrained parameter)
with torch.no_grad():
    # Get the unconstrained parameter directly
    scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")
    
    # If you want to set the constrained value to X, you need:
    # unconstrained_value = inverse_softplus(X) = log(exp(X) - 1)
    desired_constrained_value = 10.0
    unconstrained_value = torch.log(torch.exp(torch.tensor(desired_constrained_value, device=scale_param.device)) - 1)
    
    # Now modify the unconstrained parameter
    scale_param.data[0] = unconstrained_value

# Or if you want to modify the unconstrained value directly:
#with torch.no_grad():
#    scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")
#    scale_param.data[0] = your_unconstrained_value

In [27]:
scale_param

tensor([10.0000,  0.0385,  0.0440,  ...,  7.7091,  6.1614,  6.6950],
       device='cuda:0')

In [28]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<SoftplusBackward0>)

In [34]:
# Instead of storing a reference
scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

# Always access directly
with torch.no_grad():
    current_scale = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")
    current_scale.data[0] = 10

# Or if you must store a reference, make sure to update it after any param store changes
def update_scale_reference():
    global scale_param
    scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

In [35]:
update_scale_reference()

In [36]:
scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")
current_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

print("Same object?", scale_param is current_param)
print("Same values?", torch.equal(scale_param, current_param))

Same object? False
Same values? True


In [43]:
def change_item(param_store, location_index, new_value):
    pyro.get_param_store()[param_store][location_index] = new_value

    return pyro.get_param_store()[param_store]

with torch.no_grad():
    pyro.get_param_store().get_param("AutoDiagonalNormal.scale").data.copy_(change_item("AutoDiagonalNormal.scale", 0, 88))

In [44]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<SoftplusBackward0>)

In [41]:
with torch.no_grad():
    pyro.get_param_store().get_param("AutoDiagonalNormal.loc").data.copy_(change_item("AutoDiagonalNormal.loc", 0, 10))

In [42]:
pyro.get_param_store().get_param("AutoDiagonalNormal.loc")

Parameter containing:
tensor([10.0000, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', requires_grad=True)

In [49]:
import pyro

# Access the scale parameter
scale_param = pyro.get_param_store()["AutoDiagonalNormal.scale"]

# Print value before
print("Before:", scale_param.clone())  # clone to avoid referencing

# Modify it (example)
with torch.no_grad():
    scale_param[0] = 0.9

# Print value after
print("After:", pyro.get_param_store().get_param("AutoDiagonalNormal.scale").data.copy_(scale_param))


Before: tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<CloneBackward0>)
After: tensor([0.9000, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<CopyBackwards>)


In [51]:
pyro.get_param_store()["AutoDiagonalNormal.scale"]

tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<SoftplusBackward0>)

In [50]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<SoftplusBackward0>)

In [58]:
import pyro
import torch

# Get the parameter
param = pyro.get_param_store().get_param("AutoDiagonalNormal.loc")

# Modify it safely by creating a new tensor
new_param = param.clone()
new_param[0] = 10  # Your new value

# Update the parameter store
pyro.get_param_store().replace_param("AutoDiagonalNormal.loc", new_param, param)


In [60]:
pyro.get_param_store().get_param("AutoDiagonalNormal.loc")

tensor([10.0000, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', grad_fn=<CopySlices>)

In [ ]:
#def change_first_item(param_store, new_value):
#    pyro.get_param_store()[param_store][0] = new_value
#
#    return pyro.get_param_store()[param_store]

# Change the first item with no gradient tracking
#with torch.no_grad():
#    param_store["AutoDiagonalNormal.loc"].data.copy_(change_first_item("AutoDiagonalNormal.loc", 100.0))

In [ ]:
#print(param_store.keys())

In [ ]:
#print(list(param_store.keys()))
#print(param_store["AutoDiagonalNormal.scale"])

In [115]:
def change_item(param_store, location_index, new_value):
    pyro.get_param_store()[param_store][location_index] = new_value

    return pyro.get_param_store()[param_store]

def run_seu_autodiagonal_normal(location_index: int, bit_i: int, parameter_name: str="loc"):
    """Perform a bitflip at index i across every variable in the AutoDiagonalNormal guide"""

    assert bit_i in range(0, 33)
    assert parameter_name in ["loc", "scale"]
    assert location_index in range(0, len(pyro.get_param_store()[f"AutoDiagonalNormal.{parameter_name}"]))

    if parameter_name == "loc":
        param_store_name = "AutoDiagonalNormal.loc"
    elif parameter_name == "scale":
        param_store_name = "AutoDiagonalNormal.scale"

    bayesian_model.to(device)
    bayesian_model.eval()

    with torch.no_grad():
        param_dict = {}

        for name, value in pyro.get_param_store().items():
            #print(f"{name}: {value.shape}")
            #print(value)
            param_dict[name] = value.cpu().detach().numpy()

        tensor_cpu = param_dict[param_store_name]

        original_val = tensor_cpu[0]
        seu_val = bitflip_float32(original_val, bit_i)

        #pyro.get_param_store()[param_store][0] = seu_val

        print(f"Original value: {original_val}, SEU value: {seu_val}")
        #pyro.get_param_store()["AutoDiagonalNormal.loc"]

        #if param_store_name == "AutoDiagonalNormal.loc":
        #    param_store[param_store_name].data.copy_(change_item(param_store_name, location_index, seu_val))
        #elif param_store_name == "AutoDiagonalNormal.scale":
        #    param_store[param_store_name].data.copy_(change_item(param_store_name, location_index, seu_val))
        #print(f"Changed {param_store_name} at index {location_index} to {seu_val}")

        # Get the parameter
        param = pyro.get_param_store().get_param(param_store_name)

        # Modify it safely by creating a new tensor
        new_param = param.clone()
        new_param[location_index] = seu_val  # Your new value

        # Update the parameter store
        if parameter_name == "loc":
            pyro.get_param_store().replace_param(param_store_name, new_param, param)
        elif parameter_name == "scale":
            pyro.get_param_store().__setitem__(param_store_name, new_param)
        #pyro.get_param_store()[param_store_name]

In [116]:
param_store["AutoDiagonalNormal.scale"]

tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<SoftplusBackward0>)

In [117]:
param_store["AutoDiagonalNormal.loc"]

Parameter containing:
tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', requires_grad=True)

In [ ]:
#run_seu_autodiagonal_normal(location_index= 0, bit_i=2, parameter_name="loc")

Original value: 3.148340940475464, SEU value: 5.807663958573292e+19


In [118]:
param_store["AutoDiagonalNormal.loc"]

Parameter containing:
tensor([ 3.1483, -2.4763, -1.0711,  ..., -2.4452,  4.6454,  1.5156],
       device='cuda:0', requires_grad=True)

In [119]:
param_store["AutoDiagonalNormal.scale"]

tensor([0.0454, 0.0385, 0.0440,  ..., 7.7091, 6.1614, 6.6950], device='cuda:0',
       grad_fn=<SoftplusBackward0>)

In [120]:
run_seu_autodiagonal_normal(location_index= 0, bit_i=1, parameter_name="scale")

Original value: 0.04540996998548508, SEU value: 1.5452212068469636e+37


In [121]:
param_store["AutoDiagonalNormal.scale"]

tensor([1.5452e+37, 3.8508e-02, 4.3955e-02,  ..., 7.7091e+00, 6.1614e+00,
        6.6950e+00], device='cuda:0', grad_fn=<SoftplusBackward0>)

In [ ]:
# Access the unconstrained scale
scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

# Change scale[0] to 10.0
with torch.no_grad():
    scale_param.data[0] = torch.log(torch.exp(torch.tensor(10.0, device=scale_param.device)) - 1)

# Confirm the change
print("New scale[0]:", F.softplus(scale_param)[0].item())

In [ ]:
scale_param

In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

In [ ]:
scale_param

In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

In [ ]:
with torch.no_grad():
    scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")
    # Remember that scale is stored in unconstrained form (inverse softplus)
    scale_param.data[0] = torch.log(torch.exp(torch.tensor(10.0, device=scale_param.device)) - 1)

In [ ]:
scale_param

In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

In [ ]:
param_store["AutoDiagonalNormal.scale"].data.copy_(scale_param.data)

In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

In [ ]:
print(f"Original constrained value: {F.softplus(pyro.get_param_store()["AutoDiagonalNormal.scale"][0]).item()}")
print(f"SEU constrained value: {F.softplus(torch.tensor(bitflip_float32(pyro.get_param_store()["AutoDiagonalNormal.scale"].cpu().detach().numpy(), 
                                                                        0))[0])}")

In [ ]:
F.softplus(torch.tensor(10.0, device=scale_param.device))

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.scale"]

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.scale"]

In [ ]:
#with torch.no_grad():
#    guide._scale_unconstrained.data[0]

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.scale"]

In [ ]:
param_store = pyro.get_param_store()
run_seu_autodiagonal_normal(location_index= 0, bit_i=2, parameter_name="loc")

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.scale"]

In [ ]:
guide.loc

In [ ]:
guide.scale[0] = F.softplus(scale_param)[0].item()

In [ ]:
guide.scale

In [ ]:
scale_param

In [ ]:
F.softplus(scale_param)[0].item()

In [ ]:
""
def run_seu_autodiagonal_normal(location_index: int, bit_i: int, parameter_name: str="loc"):
    """Perform a bitflip at index i across every variable in the AutoDiagonalNormal guide"""

    assert bit_i in range(0, 33)
    assert parameter_name in ["loc", "scale"]
    assert location_index in range(0, len(pyro.get_param_store()[f"AutoDiagonalNormal.{parameter_name}"]))

    if parameter_name == "loc":
        param_store_name = "AutoDiagonalNormal.loc"
    elif parameter_name == "scale":
        param_store_name = "AutoDiagonalNormal.scale"

    bayesian_model.to(device)
    bayesian_model.eval()

    with torch.no_grad():
        # Get the unconstrained parameter directly
        param_tensor = pyro.get_param_store()[param_store_name]
        
        # Get the original unconstrained value
        original_val = param_tensor[location_index].cpu().detach().numpy()
        print(f"Original unconstrained value: {original_val}")
        seu_val = bitflip_float32(original_val, bit_i)

        print(f"Original unconstrained value: {original_val}, SEU unconstrained value: {seu_val}")
        
        if parameter_name == "scale":
            print(f"Original constrained value: {F.softplus(param_tensor[location_index]).item()}")
            print(f"SEU constrained value: {F.softplus(torch.tensor(seu_val)).item()}")
            #guide()

        # Set the unconstrained value directly
        param_tensor[location_index] = torch.tensor(seu_val, device=param_tensor.device)

In [ ]:
def change_item(param_store, location_index, new_value):
    pyro.get_param_store()[param_store][location_index] = new_value

    return pyro.get_param_store()[param_store]

def run_seu_autodiagonal_normal(location_index: int, bit_i: int, parameter_name: str="loc"):
    """Perform a bitflip at index i across every variable in the AutoDiagonalNormal guide"""

    assert bit_i in range(0, 33)
    assert parameter_name in ["loc", "scale"]
    assert location_index in range(0, len(pyro.get_param_store()[f"AutoDiagonalNormal.{parameter_name}"]))

    if parameter_name == "loc":
        param_store_name = "AutoDiagonalNormal.loc"
    elif parameter_name == "scale":
        param_store_name = "AutoDiagonalNormal.scale"

    bayesian_model.to(device)
    bayesian_model.eval()

    if parameter_name == "loc":
        with torch.no_grad():
            param_dict = {}

            for name, value in pyro.get_param_store().items():
                #print(f"{name}: {value.shape}")
                #print(value)
                param_dict[name] = value.cpu().detach().numpy()

            tensor_cpu = param_dict[param_store_name]

            original_val = tensor_cpu[0]
            seu_val = bitflip_float32(original_val, bit_i)

            #pyro.get_param_store()[param_store][0] = seu_val

            print(f"Original value: {original_val}, SEU value: {seu_val}")
            #pyro.get_param_store()["AutoDiagonalNormal.loc"]

            if parameter_name == "scale":
                param_tensor = pyro.get_param_store()[param_store_name]
                print(f"Original constrained value: {F.softplus(param_tensor[location_index]).item()}")
                print(f"SEU constrained value: {F.softplus(torch.tensor(seu_val)).item()}")

            if param_store_name == "AutoDiagonalNormal.loc":
                param_store[param_store_name].data.copy_(change_item(param_store_name, location_index, seu_val))
            elif param_store_name == "AutoDiagonalNormal.scale":
                param_store[param_store_name].data.copy_(change_item(param_store_name, location_index, seu_val))

    elif parameter_name == "scale":
        with torch.no_grad():
            scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")
            param_dict = {}

            for name, value in pyro.get_param_store().items():
                #print(f"{name}: {value.shape}")
                #print(value)
                param_dict[name] = value.cpu().detach().numpy()

            tensor_cpu = param_dict[param_store_name]

            original_val = tensor_cpu[0]
            seu_val = bitflip_float32(original_val, bit_i)
            print(f"Original value: {original_val}, SEU value: {seu_val}")
            scale_param.data.copy_(change_item(param_store_name, location_index, seu_val))
        #pyro.get_param_store()[param_store_name]

In [ ]:
#override first item of guide.scale, focus on changing the item directly in the param store, without using

In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.loc")

In [ ]:
#run_seu_autodiagonal_normal(location_index= 0, bit_i=1, parameter_name="loc")
run_seu_autodiagonal_normal(location_index= 0, bit_i=1, parameter_name="loc")

In [ ]:
#scale_param = pyro.get_param_store().get_param("AutoDiagonalNormal.scale")
#scale_param[0] += 10.0  # In-place modification

# Now run the guide — it will use the modified scale
#guide = AutoDiagonalNormal(bayesian_model).to(device)

## After Bitflip

In [122]:
changed = not torch.equal(pyro.get_param_store()["AutoDiagonalNormal.scale"], #AFTER 
                          original_param_store["AutoDiagonalNormal.scale"], #BEFORE
                          )
print("Weights changed:", changed)

Weights changed: True


In [123]:
after_all_labels, after_all_predictions, after_all_logits, after_all_probs = predict_data_probs(bayesian_model, test_loader, num_samples=10)

Evaluating: 100%|██████████| 100/100 [00:09<00:00, 11.08it/s]


In [124]:
after_cm = confusion_matrix(after_all_labels, after_all_predictions)

In [125]:
after_accuracy = np.trace(after_cm) / np.sum(after_cm)
print(f"Accuracy from confusion matrix: {after_accuracy * 100:.6f}%")

Accuracy from confusion matrix: 11.111111%


In [126]:
#print the difference in accuracy
print(f"Accuracy difference: {(after_accuracy - accuracy)*100:.6f}%")

Accuracy difference: -63.796296%


In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.loc")

In [ ]:
pyro.get_param_store().get_param("AutoDiagonalNormal.scale")

## END OF Model Loading

In [ ]:
mendingdiemdeh

## Real Bitflip Experiment

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.loc"]

In [ ]:
# clean pyro param store
model_path = 'results_eurosat/bayesian_cnn_model_std10_100_epoch.pth'
guide_path = 'results_eurosat/bayesian_cnn_guide_std10_100_epoch_guide.pth'
pyro_param_store_path = 'results_eurosat/pyro_param_store_std10_100_epoch.pkl'

guide = AutoDiagonalNormal(bayesian_model).to(device)
pyro.get_param_store().set_state(torch.load(pyro_param_store_path,weights_only=False))

original_param_store = {}

for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")
    original_param_store[name] = torch.tensor(value.data, requires_grad=value.requires_grad)


In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.loc"]

In [ ]:
run_seu_autodiagonal_normal(location_index= 0, bit_i=1)

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.loc"]

In [ ]:
param_shapes = {
    "conv1.weight": (32, 3, 5, 5),
    "conv1.bias": (32,),
    "conv2.weight": (64, 32, 5, 5),
    "conv2.bias": (64,),
    "fc1.weight": (num_classes, 64 * 16 * 16),
    "fc1.bias": (num_classes,)
}

def _unpack(vector):
    """Unpacks flat vector into a dict of shaped tensors"""
    params = {}
    offset = 0
    for name, shape in param_shapes.items():
        size = torch.tensor(shape).prod().item()
        flat_param = vector[offset:offset + size]
        params[name] = flat_param.view(shape)
        offset += size
    return params

In [ ]:

# access the value of generator pyro.get_param_store().items()
#pyro_params = pyro.get_param_store().items()


In [ ]:
weight_dict = {}

for name, value in pyro.get_param_store().items():
    #print(f"{name}: {value.shape}")
    #print(value)
    weight_dict[name] = value.cpu().detach().numpy()

In [ ]:
weight_dict

In [ ]:
param_shapes = {
    "conv1.weight": (32, 3, 5, 5),
    "conv1.bias": (32,),
    "conv2.weight": (64, 32, 5, 5),
    "conv2.bias": (64,),
    "fc1.weight": (num_classes, 64 * 16 * 16),
    "fc1.bias": (num_classes,)
}

In [ ]:
unpacked_params_loc = {}
offset_loc = 0

for name, shape in param_shapes.items():
    size = torch.tensor(shape).prod().item()
    flat_param = weight_dict["AutoDiagonalNormal.loc"][offset_loc:offset_loc + size]
    unpacked_params_loc[name] = torch.tensor(flat_param).view(shape)
    offset_loc += size

In [ ]:
unpacked_params_scale = {}
offset_scale = 0

for name, shape in param_shapes.items():
    size = torch.tensor(shape).prod().item()
    flat_param = weight_dict["AutoDiagonalNormal.scale"][offset_scale:offset_scale + size]
    unpacked_params_scale[name] = torch.tensor(flat_param).view(shape)
    offset_scale += size

In [ ]:
torch.cat([param.view(-1) for param in unpacked_params_loc.values()]).shape
torch.cat([param.view(-1) for param in unpacked_params_scale.values()]).shape

In [ ]:
# modify the first item of unpacked_params_weights["conv1.weight"][0, 0, 0, 0]
unpacked_params_loc["conv1.weight"][0, 0, 0, 0] = 100.0
print(unpacked_params_loc["conv1.weight"][0, 0, 0, 0])

In [ ]:
# return unpacked_params back to the param store, into a flattened form
# put unpacked_params_weights to pyro param store autodiagonalnormal.loc
# put unpacked_params_bias to pyro param store autodiagonalnormal.scale
unpacked_params_loc_flat = torch.cat([param.view(-1) for param in unpacked_params_loc.values()])
unpacked_params_scale_flat = torch.cat([param.view(-1) for param in unpacked_params_scale.values()])

#for name, param in unpacked_params_loc.items():
#    pyro.get_param_store().set_param(name, param)

In [ ]:
unpacked_params_scale_flat.shape

In [ ]:
for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")
    print(value)

In [ ]:
param_store = pyro.get_param_store()
weight_loc = param_store["AutoDiagonalNormal.loc"]

In [ ]:
weight_loc

In [ ]:
def bitflip_tensor(tensor, bit_index=0):
    flat_tensor = tensor.flatten()
    raw_bytes = flat_tensor.view(torch.uint8)

    # Choose a random byte and bit to flip
    byte_idx = torch.randint(0, raw_bytes.numel(), (1,)).item()
    mask = 1 << bit_index
    raw_bytes[byte_idx] ^= mask  # Flip bit

    return tensor

In [ ]:
def flip_bit_at(tensor, index, bit_position=0):
    val = tensor.view(torch.uint8)
    val[index] ^= (1 << bit_position)
    return val.view(tensor.dtype)

In [ ]:
# clean pyro param store
model_path = 'results_eurosat/bayesian_cnn_model_std10_100_epoch.pth'
guide_path = 'results_eurosat/bayesian_cnn_guide_std10_100_epoch_guide.pth'
pyro_param_store_path = 'results_eurosat/pyro_param_store_std10_100_epoch.pkl'

guide = AutoDiagonalNormal(bayesian_model).to(device)
#guide.load_state_dict(torch.load(guide_path))

pyro.get_param_store().set_state(torch.load(pyro_param_store_path,weights_only=False))

In [ ]:
# Flip bit at index 0, bit position 0 (least significant bit)
original_tensor = param_store["AutoDiagonalNormal.loc"]
flipped_tensor = flip_bit_at(original_tensor, index=0, bit_position=7)

# Update the parameter store with the modified tensor
param_store["AutoDiagonalNormal.loc"].data.copy_(flipped_tensor)

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.loc"].mean()

In [ ]:
param_store["AutoDiagonalNormal.loc"].data.copy_(flip_bit_at(param_store["AutoDiagonalNormal.loc"], 30, 0))

In [ ]:
param_store["AutoDiagonalNormal.loc"].data.copy_(bitflip_tensor(param_store["AutoDiagonalNormal.loc"]))

In [ ]:
def flip_bit_in_tensor(tensor, bit_position=1, flip_count=1):
    flat = tensor.view(-1)
    idx = torch.randint(0, flat.numel(), (flip_count,))
    for i in idx:
        val = flat[i].item()
        int_val = np.frombuffer(np.float32(val).tobytes(), dtype=np.uint32)[0]
        flipped = int_val ^ (1 << bit_position)
        flipped_val = np.frombuffer(np.uint32(flipped).tobytes(), dtype=np.float32)[0]
        flat[i] = torch.tensor(flipped_val)
    return tensor

def inject_seu_conv_layer(layer, bit_position=10, flip_count=1):
    with torch.no_grad():
        layer.weight.data = flip_bit_in_tensor(layer.weight.data.clone(), bit_position, flip_count)

In [ ]:
with torch.no_grad():
    flip_bit_in_tensor(pyro.get_param_store()["AutoDiagonalNormal.loc"], bit_position=3, flip_count=1)

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.loc"]

## BEFORE BITFLIP

## BITFLIP

In [ ]:
#with torch.no_grad():
#    flip_bit_in_tensor(pyro.get_param_store()["AutoDiagonalNormal.loc"], bit_position=3, flip_count=1)

In [ ]:
# change the first item of pyro.get_param_store()["AutoDiagonalNormal.loc"] to 100
# pyro.get_param_store()["AutoDiagonalNormal.loc"][0] = 100.0

def change_first_item(param_store, new_value):
    pyro.get_param_store()[param_store][0] = new_value

    return pyro.get_param_store()[param_store]

# Change the first item with no gradient tracking
with torch.no_grad():
    param_store["AutoDiagonalNormal.loc"].data.copy_(change_first_item("AutoDiagonalNormal.loc", 100.0))
    

In [ ]:
pyro.get_param_store()["AutoDiagonalNormal.loc"]

In [ ]:
original_param_store["AutoDiagonalNormal.loc"]

## AFTER BITFLIP

In [ ]:
changed = not torch.equal(pyro.get_param_store()["AutoDiagonalNormal.loc"], #AFTER 
                          original_param_store["AutoDiagonalNormal.loc"], #BEFORE
                          )
print("Weights changed:", changed)

In [ ]:
#changed = not torch.equal(model_cnn.conv1.weight.data, original_weights)
#print("Weights changed:", changed)

In [ ]:
# calculate roc auc score
from sklearn.metrics import roc_auc_score

def calculate_roc_auc_score(y_true, y_pred, num_classes):
    # Convert to one-hot encoding
    y_true_one_hot = np.eye(num_classes)[y_true]
    y_pred_one_hot = np.eye(num_classes)[y_pred]
    
    # Calculate ROC AUC for each class
    roc_auc_scores = []
    for i in range(num_classes):
        if np.sum(y_true_one_hot[:, i]) > 0:  # Check if the class is present
            roc_auc = roc_auc_score(y_true_one_hot[:, i], y_pred[:, i])
            roc_auc_scores.append(roc_auc)
        else:
            roc_auc_scores.append(np.nan)  # Class not present in the test set

    return np.array(roc_auc_scores)

In [ ]:
roc_auc_scores = calculate_roc_auc_score(all_labels, all_predictions, num_classes)
print(f"ROC AUC scores: {roc_auc_scores}")

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Example data
y_true = [0, 1, 2, 2, 1]  # true labels
y_score = [
    [0.8, 0.1, 0.1],
    [0.2, 0.6, 0.2],
    [0.1, 0.2, 0.7],
    [0.1, 0.3, 0.6],
    [0.2, 0.7, 0.1]
]  # predicted probabilities from model

y_true = np.array(all_labels)
y_score = np.array(all_probs)

# Binarize labels
n_classes = y_score.shape[1]
y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))

# Compute ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all 10 ROC curves
plt.figure(figsize=(8, 6))
colors = [
    'tab:blue',      # 1
    'tab:orange',    # 2
    'tab:green',     # 3
    'tab:red',       # 4
    'tab:purple',    # 5
    'tab:brown',     # 6
    'tab:pink',      # 7
    'tab:gray',      # 8
    'tab:olive',     # 9
    'tab:cyan'       # 10
]
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i % len(colors)],
             label='ROC curve of class {0} (area = {1:0.2f})'
                   ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves for Each Class')
plt.legend(loc="lower right")
plt.grid()
plt.show()